In [28]:
import os
import importlib

import pandas as pd

from dotenv import load_dotenv
from datetime import datetime

from backtesting import Backtest

from plutus.code.api_connector import ApiConnector
from plutus.code.backtesting_results_v2 import Mutator, StrategyBacktesting

def __strip_market_hours(dataframe: pd.DataFrame) -> pd.DataFrame:

    # Intermediary column that holds time of logged OHLCV
    dataframe['time'] = dataframe['datetime'].dt.strftime('%H:%M:%S')

    # Filter out only those rows that fall into valid market hours
    market_hours_only = dataframe[
        ('09:30:00' <= dataframe['time']) & (dataframe['time'] <= '16:00:00')
    ]

    # Drop intermediary column
    market_hours_only = market_hours_only.drop(columns=['time'])

    return market_hours_only


load_dotenv()

# Configuration for api connector
ticker = 'SPY'
start_date = '2007-01-03-00:00'
end_date = '2012-12-29-00:00'
# start_date = '2023-04-28-00:00'
# end_date = '2023-06-01-00:00'

window_size = 125

# Configure api connector
api_connector = ApiConnector()
api_connector.ticker = ticker
api_connector.start_date = start_date
api_connector.end_date = end_date

# Request or read and prepare
dataframe = api_connector.request_daily_prices()
dataframe = __strip_market_hours(dataframe)

print(dataframe)

# Initialize strategy
strategy_file = 'custom_long'
strategy_name = 'CustomLongStrategy'

strategy_module = importlib.import_module(f'plutus.code.strategies.{strategy_file}')
strategy_class = getattr(strategy_module, strategy_name)
strategy = strategy_class()

# Process
strategy.consume(ticker, dataframe, window_size)
strategy.calculate()
strategy.mark_signals()
strategy.clean()

print(strategy.dataframe)

# Calculate signal ratio
d = strategy.dataframe
sig_ratio = len(d[d['action'] == 1.0]) / len(d) * 100

print(f'Signal Ratio: {sig_ratio}')

# Write to file system
strategy.dataframe.to_csv(os.getcwd() + f'/../outputs/{ticker}:{strategy_name}.csv')

# Prepare dataframe for backtesting
input_file = 'SPY:CustomLongStrategy.csv'
mutator = Mutator()
mutator.dataframe = pd.read_csv(f'../outputs/{input_file}')
mutator.is_local = True
mutator.prepare()


# NOTE: specifically with SPY data, there is a single faulty data point
# that is 1000x times higher than the range - a typo in the data
# Therefore, we slice it out, since it skews the chart
start = datetime.strptime('2009-05-20 00:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2009-05-21 00:00:00', '%Y-%m-%d %H:%M:%S')

faulty_bars = mutator.dataframe.loc[start:end]
mutator.dataframe.drop(index=faulty_bars.index, inplace=True)

StrategyBacktesting.window_size = 180
StrategyBacktesting.stop_loss_level = 0.02
StrategyBacktesting.take_profit_level = 0.01
StrategyBacktesting.signal_threshold = 1.0

bt = Backtest(mutator.dataframe, StrategyBacktesting, cash=1000)
stats = bt.run()
print(stats)

file_name = f'./{ticker}-2007-2012.html'

bt.plot(
    plot_return=True,
    smooth_equity=False,
    plot_equity=False,
    superimpose=False,
    show_legend=False,
    filename=file_name
)

Price for SPY are available on disk. Reading
                   datetime     close    high       low      open   volume
126     2007-01-03 09:30:00  142.0200  142.31  141.9300  142.2500  1317000
127     2007-01-03 09:31:00  142.1600  142.30  141.9891  142.0200   302000
128     2007-01-03 09:32:00  142.2300  142.23  142.0800  142.1500   498200
129     2007-01-03 09:33:00  142.2500  142.28  142.1400  142.1909   242800
130     2007-01-03 09:34:00  142.1891  142.30  142.1700  142.2500   258500
...                     ...       ...     ...       ...       ...      ...
1074622 2012-12-28 15:56:00  140.2600  140.26  140.1200  140.1760  1313183
1074623 2012-12-28 15:57:00  140.2300  140.33  140.2300  140.2600   834731
1074624 2012-12-28 15:58:00  140.1700  140.26  140.1400  140.2400  1786214
1074625 2012-12-28 15:59:00  139.9500  140.20  139.9100  140.1700  4200784
1074626 2012-12-28 16:00:00  139.5100  139.95  139.5100  139.9400  2179181

[587222 rows x 6 columns]
         action ticker      

/Users/stanislavkonnov/dev/success-factory/stock-insiders/plutus/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '8H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/Users/stanislavkonnov/dev/success-factory/stock-insiders/plutus/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/Users/stanislavkonnov/dev/success-factory/stock-insiders/plutus/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(me

GridPlot(id='p4484', ...)